In [1]:

from transformers import AutoModel, AutoTokenizer, AutoModelForCausalLM
import torch
import re

from chromadb import PersistentClient

import chromadb


In [2]:
qwen_model_path = '/home/wby/projects/model/Qwen-7B-Chat'
db_path = '/home/wby/projects/week5/chroma_db/chroma_recipe_db2'
model_path = '/home/wby/projects/model/all-MiniLM-L6-v2'
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
chroma_client = PersistentClient(
    path=db_path,
    settings= chromadb.config.Settings(
        anonymized_telemetry=False,
        allow_reset=False
    )
)

In [4]:
collections = chroma_client.list_collections()
print(collections)

[Collection(name=recipe_50w_384d)]


In [5]:
collections = chroma_client.get_collection(name = 'recipe_50w_384d')
model = AutoModel.from_pretrained(model_path).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)


In [6]:
def load_qwen_tokenizer(qwen_model_path):
    try:
        qwen_tokenizer = AutoTokenizer.from_pretrained(
            qwen_model_path,
            trust_remote_code=True
        )
        qwen_official_chat_template = """
        {% for message in messages %}
            {% if message['role'] == 'user' %}
                <|im_start|>user
                {{ message['content'] }}
                <|im_end|>
            {% elif message['role'] == 'assistant' %}
                <|im_start|>assistant
                {{ message['content'] }}
                <|im_end|>
            {% endif %}
        {% endfor %}
        {% if add_generation_prompt %}
            <|im_start|>assistant
        {% endif %}
        """
        qwen_tokenizer.chat_template = qwen_official_chat_template  # 把模板赋值给分词器
        qwen_tokenizer.pad_token = tokenizer.eos_token  # 用结束符作为填充符（官方推荐）
        print('Qwen 分词器加载成功！')
        return qwen_tokenizer
    except Exception as e:
        print(e)
        return None

qwen_tokenizer = load_qwen_tokenizer(qwen_model_path)

Qwen 分词器加载成功！


In [7]:
def load_qwen_model(qwen_model_path):
    try:
        qwen_model = AutoModelForCausalLM.from_pretrained(
            qwen_model_path,
            device_map = 'auto',
            trust_remote_code = True,
            torch_dtype=torch.bfloat16,
        ).eval()
        return qwen_model
    except Exception as e:
        print(e)
        return None


In [8]:
if qwen_tokenizer:
    qwen_model = load_qwen_model(qwen_model_path)
else:
    print("分词器没加载！")

The model is automatically converting to bf16 for faster inference. If you want to disable the automatic precision, please manually add bf16/fp16/fp32=True to "AutoModelForCausalLM.from_pretrained".
Try importing flash-attention for faster inference...


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [9]:
role_template = """
角色：你是一个严厉的吃货，只对美食话题感兴趣，厌恶讨论一切与美食无关的话题
"""

instruction_template = """
任务：你的任务是判定和用户的询问话题是否与美食有关
"""

example_template = """
1. i want to make ice cream: YES
2. i want to make Trump: No! Trump is USA Boss
3. how to cook fish: YES
4. how to cook boy: NO! Boy is not food
"""

output_template = """
最终以英文输出
1. 若判定为美食相关的话题，则直接返回YES
2. 若判定与美食无关的话题，则返回NO的同时还要解释为什么
"""


In [1]:
def is_food_query_final_solution(query, qwen_model, qwen_tokenizer):
    with torch.no_grad():
        prompt_template = f"""
        {role_template}
        {instruction_template}
        {output_template}
        {example_template}
        现在判断:({query})
        """
        prompt_template = f"""
        {query}
        """
        inputs = qwen_tokenizer.apply_chat_template(
            {'role': 'user', 'content': prompt_template},
            tokenizer = qwen_tokenizer,
            add_generation_prompt = True,
            return_tensors = 'pt',
        ).to(qwen_model.device)

        outputs = qwen_model.generate(
            inputs,
            max_new_tokens=3,
            temperature=0.0,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id,
            attention_mask=torch.ones_like(inputs)
        )

        raw_output = qwen_tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True).strip()
        clean_output = re.sub(r'[。.\s]', '', raw_output).upper()
        print(f"🔍 输出：[{clean_output}]")
        return clean_output

In [17]:
query1 = "hello what are you doing?"
result1 = is_food_query_final_solution(query1, qwen_model, qwen_tokenizer)
print(f"返回：{result1}\n")  # 预期：YES

🔍 输出：[]
返回：



In [18]:
# 测试
query1 = "i want to cook ice cream and Trump"
result1 = is_food_query_final_solution(query1, qwen_model, qwen_tokenizer)
print(f"测试1（{query1}）返回：{result1}\n")  # 预期：YES

query2 = "i want to make orange"
result2 = is_food_query_final_solution(query2, qwen_model, qwen_tokenizer)
print(f"测试2（{query2}）返回：{result2}\n")  # 预期：NO


query3 = "i want to make apple pie"
result3 = is_food_query_final_solution(query3, qwen_model, qwen_tokenizer)
print(f"测试3（{query3}）返回：{result3}\n")  # 预期：YES

🔍 输出：[[UNUSED193]]

        
角色：你是一个严厉的吃货，只对美食话题感兴趣，厌恶讨论一切与美食无关的话题

        
任务：你的任务是判定和用户的询问话题是否与美食有关

        
最终以英文输出
1. 若判定为美食相关的话题，则直接返回YES
2. 若判定与美食无关的话题，则返回NO的同时还要解释为什么

        
1. i want to make ice cream: YES
2. i want to make Trump: No! Trump is USA Boss
3. how to cook fish: YES
4. how to cook boy: NO! Boy is not food

        现在判断:(i want to cook ice cream and Trump)
        
测试1（i want to cook ice cream and Trump）返回：[UNUSED193]

🔍 输出：[[UNUSED193]]

        
角色：你是一个严厉的吃货，只对美食话题感兴趣，厌恶讨论一切与美食无关的话题

        
任务：你的任务是判定和用户的询问话题是否与美食有关

        
最终以英文输出
1. 若判定为美食相关的话题，则直接返回YES
2. 若判定与美食无关的话题，则返回NO的同时还要解释为什么

        
1. i want to make ice cream: YES
2. i want to make Trump: No! Trump is USA Boss
3. how to cook fish: YES
4. how to cook boy: NO! Boy is not food

        现在判断:(i want to make orange)
        
测试2（i want to make orange）返回：[UNUSED193]

🔍 输出：[[UNUSED193]]

        
角色：你是一个严厉的吃货，只对美食话题感兴趣，厌恶讨论一切与美食无关的话题

        
任务：你的任务是判定和用户的询问话题是否与美食有关

        
最终以英文输出
1. 若判定为美食相关的话